In [1]:
from nipype.interfaces.fsl import preprocess
from nipype.interfaces.fsl import ImageMaths
from nilearn import plotting
%matplotlib inline
from niwidgets import NiftiWidget
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
from IPython.display import Javascript
import nilearn.plotting as nip#for using nilearn plot functions
from nilearn.image.image import mean_img,load_img

191020-14:43:29,588 nipype.utils INFO:
	 Running nipype version 1.3.0-dev (latest: 1.2.3)


In [2]:
results_path = "../../results/output/preproc/*"

In [3]:
list_subjs = !ls -d $results_path | grep session

In [4]:
def run_cells_below():
    display(Javascript("Jupyter.notebook.execute_cell_range(6,10)"))

In [5]:
subj_id_selector = widgets.Select(
    options=list_subjs,
    # value='C575',
    # rows=10,
    description='Subject Id:',
    disabled=False
)
#subj_id_selector.on_trait_change(run_cells_below)
display(subj_id_selector)

Select(description='Subject Id:', options=('../../results/output/preproc/_session_id_1_subject_id_C021', '../.…

In [24]:
mni_template = "../../datos/TEMPLATES/MNI152_T1_1mm.nii.gz"
mni_3mm_template = "../../datos/TEMPLATES/MNI152_T1_3mm.nii.gz"
cbl_template = "../../datos/TEMPLATES/Cerebellum-MNIflirt-maxprob-thr0-1mm.nii.gz"
cbl_template_prob = "../../datos/TEMPLATES/Cerebellum-MNIflirt-prob-2mm.nii.gz"
neck_filename = subj_id_selector.value+"/Neck_remover/mprage_ROI.nii.gz"
skull_filename = subj_id_selector.value+"/skull_strip/mprage_ROI_flirt_brain.nii.gz"
bold_filename = subj_id_selector.value+"/applyTransFunc/f1_trim_st_mcf_trans.nii.gz"
print(subj_id_selector.value)

../../results/output/preproc/_session_id_1_subject_id_T078


In [8]:
maths = ImageMaths(in_file=cbl_template, op_string= '-bin')
maths.verbose = True
bin_result = maths.run()

In [9]:
anat_widget = NiftiWidget(bin_result.outputs.out_file)
anat_widget.nifti_plotter()

<Figure size 432x288 with 0 Axes>

interactive(children=(IntSlider(value=90, continuous_update=False, description='x', max=181), IntSlider(value=…

In [10]:
cbl_img = load_img(bin_result.outputs.out_file)

In [11]:
print(cbl_img.header["dim"])

[  3 182 218 182   1   1   1   1]


In [12]:
bold_img = load_img(bold_filename)

In [13]:
print(bold_img.header["dim"])

[  4  61  73  61 158   1   1   1]


In [22]:
from nilearn.input_data import NiftiMasker,NiftiMapsMasker
masker = NiftiMasker(mask_img=bin_result.outputs.out_file, standardize=True)
fmri_masked = masker.fit_transform(bold_filename)

In [15]:
fmri_cbl = masker.inverse_transform(fmri_masked)

In [ ]:
fmri_cbl.get_data().shape

In [17]:
roi_widget = NiftiWidget(fmri_cbl)
roi_widget.nifti_plotter()

<Figure size 432x288 with 0 Axes>

interactive(children=(IntSlider(value=90, continuous_update=False, description='x', max=181), IntSlider(value=…

In [25]:
masker = NiftiMapsMasker(maps_img=cbl_template_prob, memory='nilearn_cache', memory_level=1, detrend=True,
                             verbose=10, t_r=1.94)
fmri_masked = masker.fit_transform(bold_filename)

[NiftiMapsMasker.fit_transform] loading regions from ../../datos/TEMPLATES/Cerebellum-MNIflirt-prob-2mm.nii.gz
Resampling maps
________________________________________________________________________________
[Memory] Calling nilearn.image.resampling.resample_img...
resample_img(<nibabel.nifti1.Nifti1Image object at 0x7f577ffa8898>, interpolation='continuous', target_shape=(61, 73, 61), target_affine=array([[  -3.,    0.,    0.,   90.],
       [   0.,    3.,    0., -126.],
       [   0.,    0.,    3.,  -72.],
       [   0.,    0.,    0.,    1.]]))
_____________________________________________________resample_img - 5.3s, 0.1min
________________________________________________________________________________
[Memory] Calling nilearn.input_data.base_masker.filter_and_extract...
filter_and_extract('../../results/output/preproc/_session_id_1_subject_id_T078/applyTransFunc/f1_trim_st_mcf_trans.nii.gz', 
{ 'allow_overlap': True,
  'detrend': True,
  'dtype': None,
  'high_pass': None,
  'low_p

In [26]:
fmri_cbl = masker.inverse_transform(fmri_masked)

[NiftiMapsMasker.inverse_transform] computing image from signals


In [27]:
roi_widget = NiftiWidget(fmri_cbl)
roi_widget.nifti_plotter()

<Figure size 432x288 with 0 Axes>

interactive(children=(IntSlider(value=45, continuous_update=False, description='x', max=90), IntSlider(value=5…